<p>I will be retraining the model with the top 10 features with the highest mean absolute SHAP values. That way, people using the application can play with those features to see how they will impact the house's price</p>

In [ ]:
from google.colab import files # allows us to import Kaggle csv files into colab
uploaded = files.upload() # importing the model training data

Saving train.csv to train.csv


In [ ]:
import pandas as pd # data manipulation and analysis library to store csv data
import io # allows us to manage file I/O
df = pd.read_csv(io.BytesIO(uploaded['train.csv'])) # importing csv data into pandas dataframe
# keeps top 10 features with highest mean absolute SHAP values (see Milestone 2 for proof) along with the output/label
df = df[['GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'Fireplaces', '1stFlrSF', 'GarageYrBlt', '2ndFlrSF', 'SalePrice']]
x = df.drop(['SalePrice'], axis=1) # input/data/features
y = df.SalePrice # output/label
df.head() # shows top of dataframe that contains training data and labels (ground truth)

,GrLivArea,GarageCars,TotalBsmtSF,YearBuilt,YearRemodAdd,BsmtFinSF1,Fireplaces,1stFlrSF,GarageYrBlt,2ndFlrSF,SalePrice
0,1710,2,856,2003,2003,706,0,856,2003.0,854,208500
1,1262,2,1262,1976,1976,978,1,1262,1976.0,0,181500
2,1786,2,920,2001,2002,486,1,920,2001.0,866,223500
3,1717,3,756,1915,1970,216,1,961,1998.0,756,140000
4,2198,3,1145,2000,2000,655,1,1145,2000.0,1053,250000


In [ ]:
df.min() # gets minimum values of the training data features; these are used to set the streamlit slider minimums

GrLivArea         334.0
GarageCars          0.0
TotalBsmtSF         0.0
YearBuilt        1872.0
YearRemodAdd     1950.0
BsmtFinSF1          0.0
Fireplaces          0.0
1stFlrSF          334.0
GarageYrBlt      1900.0
2ndFlrSF            0.0
SalePrice       34900.0
dtype: float64

In [ ]:
df.max() # gets maximum values of the training data features; these are used to set the streamlit slider maximums

GrLivArea         5642.0
GarageCars           4.0
TotalBsmtSF       6110.0
YearBuilt         2010.0
YearRemodAdd      2010.0
BsmtFinSF1        5644.0
Fireplaces           3.0
1stFlrSF          4692.0
GarageYrBlt       2010.0
2ndFlrSF          2065.0
SalePrice       755000.0
dtype: float64

In [ ]:
import lightgbm as ltb # gradient-boosting framework for machine learning
from sklearn.metrics import mean_squared_error # mean squared error calculating function
def objective(trial, x=x, y=y): # objective function used for optimization
  # parameter grid to search (using recommend ranges from given article)
  param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
  }
  # creates model to try hyperparameter combination sets
  model = ltb.LGBMRegressor(**param_grid)
  # fits the model to the data with a single candidate set
  model.fit(x, y)
  # generates predictions using this model
  preds = model.predict(x)
  # scores the predictions based on root mean squared error and returns it
  rmse = mean_squared_error(y, preds, squared=False)
  return rmse

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.5 MB/s eta 0:00:00


In [ ]:
import optuna # hyperparameter optimization library
study = optuna.create_study(direction="minimize") # creates study to minimize error
study.optimize(objective, n_trials=100) # uses objective function to optimize hyperparameters
print('Number of finished trials:', len(study.trials)) # prints number of finished trials
print('Best trial:', study.best_trial.params) # prints the hyperparameters for the best trial

[I 2023-04-16 23:24:09,307] A new study created in memory with name: no-name-ab44d680-d2f1-4520-b472-ad0d36f28c9b
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] min_gain_to_split is set=2.0327688585509387, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.0327688585509387
[LightGBM] [Warning] lambda_l2 is set=60, reg_lambda=0.0 will be ignored. Current value: lambda_l2=60
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=3700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3700
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-04-16 23:24:09,745] Trial 0 finished with value: 79415.29188606751 and parameters: {'n_estimators': 10000, 'learning_rate': 0.09057230512692444, 'num_leaves': 1440, 'max_depth': 8, 'min_data_in_leaf': 3700, 'lambda_l1': 70, 'lambda_l2': 60, 'min_gain_to_split': 2.0327688585509387, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 79415.29188606751.
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=85, reg_alpha=0.0 will be ignored. Current value: lambda_l1=85
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=3.8230694265885985, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.8230694265885985
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=3700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3700
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-04-16 23:24:10,157] Trial 1 finished with value: 79415.29188606751 and parameters: {'n_estimators': 10000, 'learning_rate': 0.25395493505060823, 'num_leaves': 500, 'max_depth': 3, 'min_data_in_leaf': 3700, 'lambda_l1': 85, 'lambda_l2': 55, 'min_gain_to_split': 3.8230694265885985, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 79415.29188606751.
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=0.9778430608244835, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.9778430608244835
[LightGBM] [Warning] lambda_l2 is set=65, reg_lambda=0.0 will be ignored. Current value: lambda_l2=65
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=1200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-04-16 23:26:57,217] Trial 98 finished with value: 79415.29188606751 and parameters: {'n_estimators': 10000, 'learning_rate': 0.06394113015744585, 'num_leaves': 2540, 'max_depth': 12, 'min_data_in_leaf': 1200, 'lambda_l1': 70, 'lambda_l2': 65, 'min_gain_to_split': 0.9778430608244835, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 85 with value: 24242.852181126545.
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=65, reg_alpha=0.0 will be ignored. Current value: lambda_l1=65
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_gain_to_split is set=1.4844807374290072, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.4844807374290072
[LightGBM] [Warning] lambda_l2 is set=50, reg_lambda=0.0 will be ignored. Current value: lambda_l2=50
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=700
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-04-16 23:26:58,123] Trial 99 finished with value: 79415.29188606751 and parameters: {'n_estimators': 10000, 'learning_rate': 0.053259486739086426, 'num_leaves': 2520, 'max_depth': 11, 'min_data_in_leaf': 700, 'lambda_l1': 65, 'lambda_l2': 50, 'min_gain_to_split': 1.4844807374290072, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 85 with value: 24242.852181126545.


Number of finished trials: 100
Best trial: {'n_estimators': 10000, 'learning_rate': 0.08094073979237741, 'num_leaves': 2440, 'max_depth': 12, 'min_data_in_leaf': 200, 'lambda_l1': 70, 'lambda_l2': 60, 'min_gain_to_split': 0.7677142894385782, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.8}


In [55]:
# creating model with aforementioned parameter values
model = ltb.LGBMRegressor(n_estimators=10000, learning_rate=0.08094073979237741, num_leaves=2440, max_depth=12, min_data_in_leaf=200, lambda_l1=70, lambda_l2=60, min_gain_to_split=0.7677142894385782, bagging_fraction=0.7, bagging_freq=1, feature_fraction=0.8)
# training model using train data
model.fit(x, y)

[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=0.7677142894385782, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.7677142894385782
[LightGBM] [Warning] lambda_l2 is set=60, reg_lambda=0.0 will be ignored. Current value: lambda_l2=60
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


LGBMRegressor(bagging_fraction=0.7, bagging_freq=1, feature_fraction=0.8,
              lambda_l1=70, lambda_l2=60, learning_rate=0.08094073979237741,
              max_depth=12, min_data_in_leaf=200,
              min_gain_to_split=0.7677142894385782, n_estimators=10000,
              num_leaves=2440)

In [ ]:
# saves model so it can be deployed on streamlit
model.booster_.save_model('model.txt')
# downloads file containing model
files.download('model.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>